### Hyperparameter tuning of Deep learning models in Keras

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend

In [ ]:
# Import libraries for Convolutional network models
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 5.9MB/s 


In [3]:
(img_train, y_train), (img_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# Normalize pixel values between 0 and 1
x_train = img_train.astype('float32') / 255.0
x_test = img_test.astype('float32') / 255.0

img_rows, img_cols = 28, 28
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [5]:
# define the modelef model_builder(hp):
  model = keras.Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape,padding='same'))   
  # Tune the number of layers for the Conv2D layers
  hp_layers = hp.Int('num_layers_conv', min_value = 2, max_value = 10)
  for layer in range(hp_layers):
        model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 64-512
  hp_units = hp.Int('units', min_value = 64, max_value = 512)
  model.add(Dense(units = hp_units, activation='relu'))
  model.add(Dropout(0.2))  
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

In [6]:
# Instantiate the tuner and perform hypertuning
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 5,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [7]:
#Run the hyperparameter search. The arguments for the search method are the same as those used for tf.keras.model.fit in addition to the callback above.
%time tuner.search(x_train, y_train, epochs = 5, validation_data = (x_test, y_test))

Trial 10 Complete [00h 00m 50s]
val_accuracy: 0.9162999987602234

Best val_accuracy So Far: 0.9272000193595886
Total elapsed time: 00h 08m 47s
INFO:tensorflow:Oracle triggered exit
CPU times: user 6min 31s, sys: 2min 8s, total: 8min 39s
Wall time: 8min 47s


In [9]:
model = tuner.get_best_models()

In [10]:
model[0].summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0